In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx

# Данные аминокислот и их числовые значения
amino_acid_values = {
    'A': 0.89, 'R': -0.96, 'N': -0.86, 'D': -0.98, 'C': 0.76,
    'Q': -1.0, 'E': -0.89, 'G': 0.09, 'H': -0.67, 'I': 0.99,
    'L': 0.84, 'K': -0.93, 'M': 0.22, 'F': -0.67, 'P': 0.0,
    'S': 0.67, 'T': -0.75, 'W': 0.99, 'Y': 0.84, 'V': 0.84
}

# Генерация всех возможных комбинаций длины 3
from itertools import product
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
all_combinations = [''.join(p) for p in product(amino_acids, repeat=3)]
numerical_combinations = np.array([[amino_acid_values[aa] for aa in combo] for combo in all_combinations])

# Функция для расчета "функциональности"
def calculate_functionality(vector):
    weights = np.array([0.4, 0.3, 0.3])
    functionality = (np.sum(weights * vector) + 0.1*(vector[0]*vector[1]) - 0.05*(vector[1]*vector[2]))
    return 5 + 5 * np.tanh(functionality)

# Для построения дерева разбиения кластеров
cluster_tree = []  # Список кортежей (parent, child)
cluster_id_counter = 0  # Глобальный счетчик для уникальных id
cluster_id_map = {}  # mapping: (iteration, local_cluster_num) -> global_id

# Для хранения данных кластеров на каждом шаге для последующей визуализации
all_cluster_data = []

# Инициализация кластеров
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(numerical_combinations)

# Присваиваем id начальным кластерам
for i in range(n_clusters):
    cluster_id_map[(0, i)] = cluster_id_counter
    cluster_id_counter += 1

# Функция для получения данных кластера с 30 последовательностями
def get_cluster_data(cluster_labels, n_samples=100, min_cluster_size=10):
    cluster_data = []
    for cluster_num in range(np.max(cluster_labels)+1):
        cluster_indices = np.where(cluster_labels == cluster_num)[0]
        if len(cluster_indices) < min_cluster_size:
            continue  # Пропускаем слишком маленькие кластеры
        # Сначала только первые n_samples последовательностей будут с рассчитанным фитнесом
        selected_indices = np.random.choice(cluster_indices, size=min(n_samples, len(cluster_indices)), replace=False)
        sequences = []
        for idx in selected_indices:
            seq = all_combinations[idx]
            vec = numerical_combinations[idx]
            func = calculate_functionality(vec)
            sequences.append({'sequence': seq, 'vector': vec, 'functionality': func, 'index': idx})
        avg_func = np.mean([s['functionality'] for s in sequences])
        cluster_data.append({
            'cluster': cluster_num,
            'avg_functionality': avg_func,
            'sequences': sequences,
            'indices': cluster_indices
        })
    return cluster_data

cluster_data = get_cluster_data(clusters)

fitness_history = []  # Для хранения фитнеса на каждом шаге
num_clusters_history = []  # Для хранения количества кластеров на каждом шаге
func_distributions = []   # Для хранения распределения функциональности на каждом шаге

# Итеративное разделение кластеров
max_iterations = 8
FITNESS_THRESHOLD = 7.0  # Порог для обязательного деления
for iteration in tqdm(range(max_iterations)):
    new_cluster_labels = np.copy(clusters)
    current_cluster_offset = 0
    
    # Сохраняем количество кластеров и распределение функциональности
    num_clusters_history.append(len(cluster_data))
    func_distributions.append([c['avg_functionality'] for c in cluster_data])
    print(f"\nШаг {iteration+1}: количество кластеров = {len(cluster_data)}")
    # Вывод фитнеса кластеров на текущем шаге
    print(f"Фитнес (средняя функциональность) по кластерам:")
    fitness_step = []
    for cluster in cluster_data:
        print(f"  Кластер {cluster['cluster']}: средняя функциональность = {cluster['avg_functionality']:.2f}, количество последовательностей = {len(cluster['indices'])}")
        fitness_step.append(cluster['avg_functionality'])
    fitness_history.append(fitness_step)

    new_id_map = {}
    for h, cluster in enumerate(cluster_data):
        C_k = len(cluster['sequences'])
        sum_y_k = sum([s['functionality'] for s in cluster['sequences']])
        sum_all_y = sum([sum([s['functionality'] for s in c['sequences']]) for c in cluster_data])
        sum_all_C = sum([len(c['sequences']) for c in cluster_data])
        P_k = (sum_y_k / C_k) / (sum_all_y / sum_all_C)
        avg_funcs = [np.mean([s['functionality'] for s in c['sequences']]) for c in cluster_data]
        if h == np.argmax(avg_funcs):
            K_h = len(cluster_data)
            L_k = max(1, K_h - sum([int(P_k * K_h) + 1 for j in range(len(cluster_data)) if j != h]))
        else:
            L_k = max(1, int(P_k * len(cluster_data)) + 1)
        n_samples = len(cluster['sequences'])
        # Обязательное деление, если фитнес выше порога и есть что делить
        if cluster['avg_functionality'] > FITNESS_THRESHOLD and n_samples > 1:
            L_k = max(L_k, 2)
        if n_samples <= 1:
            L_k = 1
        else:
            L_k = min(L_k, n_samples - 1)
        if L_k > 1:
            cluster_vectors = np.array([s['vector'] for s in cluster['sequences']])
            sub_kmeans = KMeans(n_clusters=L_k, random_state=42)
            sub_clusters = sub_kmeans.fit_predict(cluster_vectors)
            # Для каждого подклстера создаём новую вершину и ребро к родителю
            for sub in range(L_k):
                parent_id = cluster_id_map[(iteration, h)]
                child_id = cluster_id_counter
                cluster_tree.append((parent_id, child_id))
                new_id_map[(iteration+1, current_cluster_offset+sub)] = child_id
                cluster_id_counter += 1
            # Присваиваем каждому элементу уникальный id подкластера
            predicted_labels = sub_kmeans.predict(numerical_combinations[cluster['indices']])
            for i, idx in enumerate(cluster['indices']):
                new_cluster_labels[idx] = new_id_map[(iteration+1, current_cluster_offset + predicted_labels[i])]
            current_cluster_offset += L_k
        else:
            # Если не делим, сохраняем id родителя
            parent_id = cluster_id_map[(iteration, h)]
            child_id = parent_id  # id не меняется
            cluster_tree.append((parent_id, child_id))
            new_id_map[(iteration+1, current_cluster_offset)] = child_id
            for idx in cluster['indices']:
                new_cluster_labels[idx] = child_id
            current_cluster_offset += 1
    cluster_id_map = new_id_map
    cluster_data = get_cluster_data(new_cluster_labels)
    
    # Обучение Гауссовского процесса для каждого подкластера
    for cluster in cluster_data:
        # Используем только те, у кого нет 'predicted' и они в первых 100
        known_seqs = [s for s in cluster['sequences'] if not s.get('predicted', False)]
        if len(known_seqs) >= 2:
            X_train = np.array([s['vector'] for s in known_seqs])
            y_train = np.array([s['functionality'] for s in known_seqs])
            length_scales = np.std(X_train, axis=0)
            kernel = ConstantKernel(1.0) * RBF(length_scale=length_scales, length_scale_bounds=(1e-2, 1e6))
            gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1, n_restarts_optimizer=10)
            gp.fit(X_train, y_train)
            all_vectors = numerical_combinations[cluster['indices']]
            predicted_func = gp.predict(all_vectors)
            for i, idx in enumerate(cluster['indices']):
                if idx not in [s['index'] for s in cluster['sequences']]:
                    cluster['sequences'].append({
                        'sequence': all_combinations[idx],
                        'vector': numerical_combinations[idx],
                        'functionality': predicted_func[i],
                        'index': idx,
                        'predicted': True
                    })

    clusters = new_cluster_labels

    # Обновление данных кластеров с 100 последовательностями
    cluster_data = get_cluster_data(new_cluster_labels)
    all_cluster_data.append([{ # Сохраняем копию данных для визуализации позже
        'cluster': c['cluster'],
        'sequences': [{'sequence': s['sequence'], 'vector': s['vector'], 'functionality': s['functionality'], 'index': s['index'], 'predicted': s.get('predicted', False)}
                      for s in c['sequences']]
    } for c in cluster_data])

# Финальная визуализация
plt.figure(figsize=(14, 7))
for cluster in cluster_data:
    x = [cluster['cluster']] * len(cluster['sequences'])
    y = [s['functionality'] for s in cluster['sequences']]
    plt.scatter(x, y, s=30, alpha=0.7)
plt.title('Финальное распределение функциональности по кластерам', fontsize=16)
plt.xlabel('Номер кластера', fontsize=14)
plt.ylabel('Функциональность', fontsize=14)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Визуализация дерева кластеризации (только первые 2 уровня)
G = nx.DiGraph()
G.add_edges_from(cluster_tree)

# Получаем только первые 2 уровня дерева
roots = [n for n, d in G.in_degree() if d == 0]
level1 = []
for r in roots:
    level1.extend(list(G.successors(r)))
level2 = []
for n in level1:
    level2.extend(list(G.successors(n)))
nodes_to_draw = set(roots + level1 + level2)
edges_to_draw = [(u, v) for u, v in G.edges() if u in nodes_to_draw and v in nodes_to_draw]
H = G.edge_subgraph(edges_to_draw).copy()

plt.figure(figsize=(18, 8))
try:
    pos = nx.nx_pydot.graphviz_layout(H, prog='dot')
    nx.draw(H, pos, with_labels=True, arrows=False, node_size=500, font_size=8)
    plt.title('Дерево разбиения кластеров (первые 2 уровня)')
    plt.show()
except Exception as e:
    print('Ошибка при построении дерева кластеров:', e)
    print('Убедитесь, что установлен Graphviz и его путь добавлен в PATH.')

# Перенумерация кластеров для последовательности
for new_idx, cluster in enumerate(cluster_data):
    cluster['old_cluster'] = cluster['cluster']
    cluster['cluster'] = new_idx

# График фитнеса по шагам
plt.figure(figsize=(10, 6))
for i, fitness_step in enumerate(fitness_history):
    plt.plot([i+1]*len(fitness_step), fitness_step, 'o', label=f'Шаг {i+1}' if i==0 else None)
plt.xlabel('Шаг разбиения')
plt.ylabel('Средняя функциональность кластеров')
plt.title('Динамика фитнеса кластеров на каждом шаге')
plt.grid(True, alpha=0.3)
plt.show()

# Вывод информации о кластерах
print("\nФинальные кластеры:")
for cluster in sorted(cluster_data, key=lambda x: -x['avg_functionality']):
    print(f"\nКластер {cluster['cluster']} (старый номер: {cluster['old_cluster']}):")
    print(f"Количество последовательностей: {len(cluster['indices'])}")
    print(f"Средняя функциональность: {cluster['avg_functionality']:.2f}")
    print("Примеры последовательностей (функциональность):")
    sample_seqs = np.random.choice(cluster['sequences'], size=min(5, len(cluster['sequences'])), replace=False)
    for seq in sample_seqs:
        pred_tag = "(предсказано)" if seq.get('predicted', False) else ""
        print(f"{seq['sequence']}: {seq['functionality']:.2f} {pred_tag}")
    # Вывод всех предсказанных последовательностей
    predicted_seqs = [s for s in cluster['sequences'] if s.get('predicted', False)]
    if predicted_seqs:
        print("  Последовательности с предсказанным значением функциональности:")
        for s in predicted_seqs:
            print(f"    {s['sequence']}: {s['functionality']:.2f} (предсказано)")
    num_predicted = sum(1 for s in cluster['sequences'] if s.get('predicted', False))
    num_calculated = len(cluster['sequences']) - num_predicted
    print(f"  Всего последовательностей: {len(cluster['sequences'])}")
    print(f"  Рассчитано по формуле: {num_calculated}")
    print(f"  Предсказано моделью: {num_predicted}")

# Вывод общего количества кластеров
print("\nОбщее количество кластеров:", len(cluster_data))

# Вывод кластеров с высокой функциональностью (средняя функциональность > 7.0)
high_functionality_clusters = [cluster for cluster in cluster_data if cluster['avg_functionality'] > 7.0]
print("\nКластеры с высокой функциональностью (средняя функциональность > 7.0):")
for cluster in high_functionality_clusters:
    print(f"Кластер {cluster['cluster']} (старый номер: {cluster['old_cluster']}): средняя функциональность = {cluster['avg_functionality']:.2f}")

# После всех шагов: выводим сумму кластеров
print("\nСуммарное количество кластеров за все шаги:", sum(num_clusters_history))

# График: количество кластеров на каждом шаге
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(num_clusters_history)+1), num_clusters_history, marker='o')
plt.xlabel('Шаг разбиения')
plt.ylabel('Количество кластеров')
plt.title('Количество кластеров на каждом шаге')
plt.grid(True, alpha=0.3)
plt.show()

# График: распределение функциональности по кластерам на каждом шаге
plt.figure(figsize=(12, 6))
for i, func_dist in enumerate(func_distributions):
    plt.hist(func_dist, bins=20, alpha=0.5, label=f'Шаг {i+1}')
plt.xlabel('Средняя функциональность кластера')
plt.ylabel('Частота')
plt.title('Распределение функциональности по кластерам на каждом шаге')
plt.legend()
plt.show()

# Построение графиков распределения функциональности для последних двух шагов
if len(all_cluster_data) >= 2:
    start_step = len(all_cluster_data) - 2
    for step_idx in range(start_step, len(all_cluster_data)):
        clusters_to_plot = all_cluster_data[step_idx]
        current_step_num = step_idx + 1
        
        plt.figure(figsize=(14, 7))
        for cluster in clusters_to_plot:
            x = [cluster['cluster']] * len(cluster['sequences'])
            y = [s['functionality'] for s in cluster['sequences']]
            plt.scatter(x, y, s=30, alpha=0.7)
        plt.title(f'Финальное распределение функциональности по кластерам (Шаг {current_step_num})', fontsize=16)
        plt.xlabel('Номер кластера', fontsize=14)
        plt.ylabel('Функциональность', fontsize=14)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# После всех шагов: выводим характеристики для каждого шага
for step, clusters_at_step in enumerate(func_distributions):
    print(f"\nШаг {step+1}: количество кластеров = {num_clusters_history[step]}")
    print("Фитнес (средняя функциональность) по кластерам:")
    # Для последнего шага используем cluster_data для количества последовательностей
    if step == len(func_distributions) - 1:
        for i, func in enumerate(clusters_at_step):
            count = len(cluster_data[i]['indices']) if i < len(cluster_data) else 'N/A'
            print(f"  Кластер {i}: средняя функциональность = {func:.2f}, количество последовательностей = {count}")
    else:
        # Для промежуточных шагов используем сохранённые данные о количестве последовательностей
        for i, func in enumerate(clusters_at_step):
            # Для шага 0 (до разбиения) используем cluster_data до первого разбиения
            if step == 0:
                count = len(get_cluster_data(clusters)[i]['indices']) if i < len(get_cluster_data(clusters)) else 'N/A'
                print(f"  Кластер {i}: средняя функциональность = {func:.2f}, количество последовательностей = {count}")
            else:
                print(f"  Кластер {i}: средняя функциональность = {func:.2f}")

# Гистограмма распределения функциональности по всем финальным кластерам
final_func = [c['avg_functionality'] for c in cluster_data]
plt.figure(figsize=(10, 5))
plt.hist(final_func, bins=20, alpha=0.7, color='royalblue')
plt.xlabel('Средняя функциональность кластера')
plt.ylabel('Частота')
plt.title('Распределение функциональности по всем финальным кластерам')
plt.grid(True, alpha=0.3)
plt.show()